In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Step 1: Data Acquisition
# Load the Energy Efficiency dataset from UCI
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/00242/ENB2012_data.xlsx"
df = pd.read_excel(url)

# Column names based on UCI dataset description
df.columns = [
    'Relative_Compactness', 'Surface_Area', 'Wall_Area', 'Roof_Area', 'Overall_Height',
    'Orientation', 'Glazing_Area', 'Glazing_Area_Distribution', 'Heating_Load', 'Cooling_Load'
]

# We'll predict 'Heating_Load' as the target variable, so we have an energy prediction task

# Step 2: Define the Methodology and Objectives
# Objective: Predict heating load (Energy Efficiency) for buildings based on input features.
X = df.drop(columns=['Heating_Load', 'Cooling_Load'])  # Features
y = df['Heating_Load']  # Target variable

# Step 3: Data Preprocessing
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Scale the data for better ML model performance
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Step 4: Use Multiple ML Methods and Validate with K-Fold Cross Validation
models = {
    'Linear Regression': LinearRegression(),
    'Random Forest Regressor': RandomForestRegressor(n_estimators=100, random_state=0),
    'XGBoost Regressor': XGBRegressor(n_estimators=100, learning_rate=0.1, random_state=0)
}

kf = KFold(n_splits=5, shuffle=True, random_state=0)
model_scores = {}

for model_name, model in models.items():
    # Cross-validation
    cv_scores = cross_val_score(model, X_train_scaled, y_train, cv=kf, scoring='neg_mean_squared_error')
    model_scores[model_name] = np.mean(np.abs(cv_scores))  # Using mean of absolute MSE scores

# Step 5: Comparing Results
# Train the models on the whole training set for final evaluation
results = {}
for model_name, model in models.items():
    model.fit(X_train_scaled, y_train)
    predictions = model.predict(X_test_scaled)
    mse = mean_squared_error(y_test, predictions)
    mae = mean_absolute_error(y_test, predictions)
    r2 = r2_score(y_test, predictions)
    results[model_name] = {'MSE': mse, 'MAE': mae, 'R2 Score': r2}

# Display results
print("Model Performance (Mean Absolute Error during Cross-Validation):")
for model_name, score in model_scores.items():
    print(f"{model_name}: {score:.2f}")

print("\nFinal Model Results (Test Set Performance):")
for model_name, metrics in results.items():
    print(f"{model_name}:")
    print(f"  MSE: {metrics['MSE']:.2f}")
    print(f"  MAE: {metrics['MAE']:.2f}")
    print(f"  R2 Score: {metrics['R2 Score']:.2f}\n")


Model Performance (Mean Absolute Error during Cross-Validation):
Linear Regression: 8.42
Random Forest Regressor: 0.30
XGBoost Regressor: 0.22

Final Model Results (Test Set Performance):
Linear Regression:
  MSE: 10.10
  MAE: 2.22
  R2 Score: 0.91

Random Forest Regressor:
  MSE: 0.34
  MAE: 0.38
  R2 Score: 1.00

XGBoost Regressor:
  MSE: 0.23
  MAE: 0.30
  R2 Score: 1.00

